In [11]:
import pandas as pd
import numpy as np

# To-do: 
* Havem't modified this notebook yet
* Need to load in my new fasta from the Genbank, then align with IMGT
* Match up with their neut data
* Also need to pick the right ebola seq

Notes:
* They tested 321, only 2 Abs failed to bind to GPdMuc, indicating that these bind to the mucin-like domain
* This means that they all should bind the overall GP
* Since I can't find the joined sepcific seqs they used for BLI, and the PlabDab has different naming for the Abs, it will make the most sense to just use the same strain GP for all seqs

In [2]:
# Q1PDC7| VGP_MABVR Envelope glycoprotein OS=Lake Victoria marburgvirus (strain Ravn-87) 
# https://rest.uniprot.org/uniprotkb/Q1PDC7.fasta
# gp = "MKTIYFLISLILIQSIKTLPVLEIASNSQPQDVDSVCSGTLQKTEDVHLMGFTLSGQKVADSPLEASKRWAFRTGVPPKNVEYTEGEEAKTCYNISVTDPSGKSLLLDPPSNIRDYPKCKTVHHIQGQNPHAQGIALHLWGAFFLYDRVASTTMYRGKVFTEGNIAAMIVNKTVHRMIFSRQGQGYRHMNLTSTNKYWTSSNETQRNDTGCFGILQEYNSTNNQTCPPSLKPPSLPTVTPSIHSTNTQINTAKSGTMNPSSDDEDLMISGSGSGEQGPHTTLNVVTEQKQSSTILSTPSLHPSTSQHEQNSTNPSRHAVTEHNGTDPTTQPATLLNNTNTTPTYNTLKYNLSTPSPPTRNITNNDTQRELAESEQTNAQLNTTLDPTENPTTGQDTNSTTNIIMTTSDITSKHPTNSSPDSSPTTRPPIYFRKKRSIFWKEGDIFPFLDGLINTEIDFDPIPNTETIFDESPSFNTSTNEEQHTPPNISLTFSYFPDKNGDTAYSGENENDCDAELRIWSVQEDDLAAGLSWIPFFGPGIEGLYTAGLIKNQNNLVCRLRRLANQTAKSLELLLRVTTEERTFSLINRHAIDFLLTRWGGTCKVLGPDCCIGIEDLSKNISEQIDKIRKDEQKEETGWGLGGKWWTSDWGVLTNLGILLLLSIAVLIALSCICRIFTKYIG"
# print(len(gp))

# Q05320|VGP_EBOZM Envelope glycoprotein OS=Zaire ebolavirus (strain Mayinga-76)
# https://rest.uniprot.org/uniprotkb/Q05320.fasta

gp = "MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSIPLGVIHNSTLQVSDVDKLVCRDKLSSTNQLRSVGLNLEGNGVATDVPSATKRWGFRSGVPPKVVNYEAGEWAENCYNLEIKKPDGSECLPAAPDGIRGFPRCRYVHKVSGTGPCAGDFAFHKEGAFFLYDRLASTVIYRGTTFAEGVVAFLILPQAKKDFFSSHPLREPVNATEDPSSGYYSTTIRYQATGFGTNETEYLFEVDNLTYVQLESRFTPQFLLQLNETIYTSGKRSNTTGKLIWKVNPEIDTTIGEWAFWETKKNLTRKIRSEELSFTVVSNGAKNISGQSPARTSSDPGTNTTTEDHKIMASENSSAMVQVHSQGREAAVSHLTTLATISTSPQSLTTKPGPDNSTHNTPVYKLDISEATQVEQHHRRTDNDSTASDTPSATTAAGPPKAENTNTSKSTDFLDPATTTSPQNHSETAGNNNTHHQDTGEESASSGKLGLITNTIAGVAGLITGGRRTRREAIVNAQPKCNPNLHYWTTQDEGAAIGLAWIPYFGPAAEGIYIEGLMHNQDGLICGLRQLANETTQALQLFLRATTELRTFSILNRKAIDFLLQRWGGTCHILGPDCCIEPHDWTKNITDKIDQIIHDFVDKTLPDQGDNDNWWTGWRQWIPAGIGVTGVIIAVIALFCICKFVF"
print(len(gp))

676


In [3]:
import pandas as pd

def load_fasta_to_dataframe(fasta_file):
    """Load a FASTA file into a pandas DataFrame."""
    # Initialize variables
    sequences = {}
    
    # Read the FASTA file
    with open(fasta_file, 'r') as file:
        sequence_id = None
        sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):  # Header line
                if sequence_id is not None:
                    sequences[sequence_id] = sequence
                sequence_id = line[1:]  # Remove '>'
                sequence = ''
            else:
                sequence += line
        # Catch the last sequence in the file
        if sequence_id is not None:
            sequences[sequence_id] = sequence
    
    # Create a DataFrame
    df = pd.DataFrame(list(sequences.items()), columns=['Header', 'Sequence'])
    df.set_index('Header', inplace=True)
    
    return df

fasta_file = './ebola-ehrhardt_genbank.fasta'
genbank_seqs = load_fasta_to_dataframe(fasta_file)

In [5]:
genbank_seqs['clone'] = genbank_seqs.index


In [6]:
genbank_seqs['clone'] = genbank_seqs.index


In [7]:
hc_df = genbank_seqs[genbank_seqs['clone'].apply(lambda x: 'heavy' in x)]
hc_df['clone'] = hc_df['clone'].apply(lambda x: x.split('clone')[1].split(' ')[1])

lc_df = genbank_seqs[genbank_seqs['clone'].apply(lambda x: 'light' in x)]
lc_df['clone'] = lc_df['clone'].apply(lambda x: x.split('clone')[1].split(' ')[1])

C:\Users\perry\AppData\Local\Temp\ipykernel_5748\4083924311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hc_df['clone'] = hc_df['clone'].apply(lambda x: x.split('clone')[1].split(' ')[1])
C:\Users\perry\AppData\Local\Temp\ipykernel_5748\4083924311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lc_df['clone'] = lc_df['clone'].apply(lambda x: x.split('clone')[1].split(' ')[1])


In [8]:
# Drop clones with multiple heavy or light chains
hc_df = hc_df.loc[hc_df['clone'].drop_duplicates(keep='first').index]
lc_df = lc_df.loc[lc_df['clone'].drop_duplicates(keep='first').index]

In [9]:
lc_df.index = lc_df['clone']
hc_df.index = hc_df['clone']

In [12]:
len(np.intersect1d(hc_df['clone'], lc_df['clone']))

245

In [3]:
plab = pd.read_csv('../paired_sequences_plab.csv', index_col=0)

In [7]:
# plab[plab['targets_mentioned'] == 'Ebola']['reference_title'].value_counts()

In [11]:
plab_abs = plab[plab['reference_title'] == 'Polyclonal and convergent antibody response to Ebola virus vaccine rVSV-ZEBOV'].copy()

In [12]:
plab_abs

,heavy_sequence,light_sequence,heavy_ID,light_ID,heavy_definition,light_definition,organism,reference_authors,reference_title,update_date,cdr_lengths,model,pairing,targets_mentioned
ID,,,,,,,,,,,,,,
QFQ61529_QFQ61681,EVQLLESGGGLVQPGGSLRLSCAASGFAFSTYAMTWVRQAPGKGLE...,EIVLTQSPATLSLSPGERATLSCRASQSVGSSLAWYQHKPGQAPRL...,QFQ61529,QFQ61681,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_8_19_6_3_10,QFQ61529_QFQ61681,Unique source,Ebola
QFQ61530_QFQ61682,QVQLVESGGGVVQPGGSLRLSCAASGFTFSNYGMHWVRQAPGKGLE...,EIVMTQSPATLSVSPGERATLSCRASQSVSSNLAWYQQKPGQAPRL...,QFQ61530,QFQ61682,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_8_15_6_3_10,QFQ61530_QFQ61682,Unique source,Ebola
QFQ61531_QFQ61683,QVQLVQSGAEAKKPGASVRVSCQTSGYTFTGHYLHWVRQAPGQGLE...,DIVMTQFPDFVAVSPGERATISCESSQSVLYQSVAWYQHKPGQPPK...,QFQ61531,QFQ61683,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_8_10_7_3_9,QFQ61531_QFQ61683,Unique source,Ebola
QFQ61532_QFQ61684,QVQLVQSGAEVKKPGASVKVSCKSSGYTFTNYYMHWVRQAPGQGLE...,QSALTQPPSVSVSPGQTARITCSGDALPKLFAYWYQQKPGQAPVLV...,QFQ61532,QFQ61684,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_8_13_6_3_11,QFQ61532_QFQ61684,Unique source,Ebola
QFQ61533_QFQ61685,QVQLQESGPGLVKPSETLSLTCIVSGGSLSSYYWSWVRQPAGKGLQ...,SYELTQSSSVSVSPGQTASITCSGDKLGDNYVCWYQQKPGQSPTLV...,QFQ61533,QFQ61685,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_7_17_6_3_9,QFQ61533_QFQ61685,Unique source,Ebola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QFQ61679_QFQ61831,QVQLQESGPGLVKPSETLSLTCTVSGGSLSTYYWTWIRQPPGKGLE...,SYELTQPPSVSVSPGQTASITCSGDKLGDEYVSWYQQKPGQSPVLV...,QFQ61679,QFQ61831,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_7_19_6_3_10,QFQ61679_QFQ61831,Unique source,Ebola
QFQ61680_QFQ61832,EVQLLESGGGLIQPGGSLTLSCTASGFTIRNYAMSWVRQAPGKGLE...,DVVMTQSPLSLPVTLGQPASISCRSSQSLLYSDGNTYLNWFQQRPG...,QFQ61680,QFQ61832,"immunoglobulin heavy chain variable region, pa...","immunoglobulin light chain variable region, pa...",Homo sapiens,"Ehrhardt,S.A.; Zehner,M.; Krahling,V.; Cohen-D...",Polyclonal and convergent antibody response to...,23-OCT-2019,8_8_8_11_3_9,QFQ61680_QFQ61832,Unique source,Ebola
6QD7_X_Z,EVQLLESGGGLVQPGGSLRLSCEASGFPLRDYAMSWVRQAPGRGLQ...,EIVLTQSPATLSLSPGERATLSCRASQSVSTYLAWYQHQPGQAPRL...,6QD7_X,6QD7_Z,Em Structure Of A Ebov-Gp Bound To 3T0331 Neut...,Em Structure Of A Ebov-Gp Bound To 3T0331 Neut...,Homo sapiens,"Ehrhardt,S.A.;Zehner,M.;Krahling,V.;Cohen-Dvas...",Polyclonal and convergent antibody response to...,01-JAN-2019,8_8_14_6_3_9,QFQ61582_QFQ61734,Xtal structure,Ebola


In [74]:
seq_df = plab_abs[['heavy_sequence', 'light_sequence']].copy()
seq_df['antigen_seq'] = gp

In [76]:
seq_df.to_csv('ebola_bornholdt_seqs_24-01-03.csv')

#### Trying to align to sequences with KD values
* It would be good to double check this at some point, and also make sure the paired sequences were correctly paired

In [18]:
binding_table = pd.read_csv('../bornholdt_binding_table_24-01-02.csv', header=[1,2], index_col=0)

In [39]:
binding_table['Zaire GP∆muc']['IgG K_{D} (M)*'].apply(lambda x: type(x) == str)

ADI-15731    True
ADI-15732    True
ADI-15733    True
ADI-15734    True
ADI-15735    True
             ... 
ADI-16059    True
ADI-16060    True
ADI-16061    True
ADI-16062    True
ADI-16063    True
Name: IgG K_{D} (M)*, Length: 321, dtype: bool

In [45]:
# Only keeping those with a binding constant for Zaire GPdMuc
# Not sure if this is correct antigen to use yet, but we will go with it

binding_table = binding_table[~pd.to_numeric(binding_table['Zaire GP∆muc']['IgG K_{D} (M)*'], errors='coerce').isna()]

In [46]:
plab = pd.read_csv('../paired_sequences_plab.csv', index_col=0)

In [56]:
plab_abs = plab[plab['reference_title'] == 'Rapid and large-scale isolation of potent neutralizing antibodies from a survivor of the 2014 Ebola virus outbreak'].copy()

In [65]:
id_table = pd.read_csv('./bornholdt_ID_table.csv', index_col=1, header=2)

In [67]:
plab_abs.head(3)

,heavy_sequence,light_sequence,heavy_ID,light_ID,heavy_definition,light_definition,organism,reference_authors,reference_title,update_date,cdr_lengths,model,pairing,targets_mentioned
ID,,,,,,,,,,,,,,
AMK70035_AMK70036,QVQLVQSGGGLVQPGRSLRLSCTSSGFTFGDYVMSWVRQAPGKGLE...,DIRVTQSPSSLSASVGDRVTITCRASQSISTYLNWYQQKPGKAPKV...,AMK70035,AMK70036,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_10_19_6_3_9,AMK70035_AMK70036,Unique source,Ebola
AMK70037_AMK70038,QVQLVQSGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,DIVLTQSPSTLSASVGDRVTITCRASQTISRWLAWYQQKPGKAPKL...,AMK70037,AMK70038,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_15_6_3_8,AMK70037_AMK70038,Unique source,Ebola
AMK70039_AMK70040,EVQLLESGGGLVKPGGSLRLSCAASGFTFSDYTMNWVRQAPGKGLE...,DIQMTQSPSSLSVSVGDRVTITCRASQSISSFLNWYQQKPGKAPKL...,AMK70039,AMK70040,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_17_6_3_9,AMK70039_AMK70040,Unique source,Ebola


In [66]:
id_table

,Clone name,VL
VH,,
KU602083,ADI-15731,KU602084
KU602085,ADI-15732,KU602086
KU602087,ADI-15733,KU602088
KU602089,ADI-15734,KU602090
KU602091,ADI-15735,KU602092
...,...,...
KU602715,ADI-16059,KU602716
KU602717,ADI-16060,KU602718
KU602719,ADI-16061,KU602720


In [60]:
plab_abs

,heavy_sequence,light_sequence,heavy_ID,light_ID,heavy_definition,light_definition,organism,reference_authors,reference_title,update_date,cdr_lengths,model,pairing,targets_mentioned
ID,,,,,,,,,,,,,,
AMK70035_AMK70036,QVQLVQSGGGLVQPGRSLRLSCTSSGFTFGDYVMSWVRQAPGKGLE...,DIRVTQSPSSLSASVGDRVTITCRASQSISTYLNWYQQKPGKAPKV...,AMK70035,AMK70036,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_10_19_6_3_9,AMK70035_AMK70036,Unique source,Ebola
AMK70037_AMK70038,QVQLVQSGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLE...,DIVLTQSPSTLSASVGDRVTITCRASQTISRWLAWYQQKPGKAPKL...,AMK70037,AMK70038,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_15_6_3_8,AMK70037_AMK70038,Unique source,Ebola
AMK70039_AMK70040,EVQLLESGGGLVKPGGSLRLSCAASGFTFSDYTMNWVRQAPGKGLE...,DIQMTQSPSSLSVSVGDRVTITCRASQSISSFLNWYQQKPGKAPKL...,AMK70039,AMK70040,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_17_6_3_9,AMK70039_AMK70040,Unique source,Ebola
AMK70041_AMK70042,QVQLQQWGAGLLKPSETLSLTCAVYGGSLSGYYWSWIRQPPGKGLE...,DIQLTQSPLSLPVTPGEPASISCRSSQSLLHSNAYNYLDWYLQKPG...,AMK70041,AMK70042,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_7_17_11_3_9,AMK70041_AMK70042,Unique source,Ebola
AMK70043_AMK70044,EVQLVESGGGLVKPGGSLRLSCEASGFTFTNAWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQSISRSLNWYQQKIGKAPKL...,AMK70043,AMK70044,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_10_20_6_3_9,AMK70043_AMK70044,Unique source,Ebola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMK70667_AMK70668,EVQLLESGGGLAQPGGSLRLSCAASGFTFSSYGMSWVRQAPGKGLE...,QSVVTQPPSVSAAPGQKVTISCSGSTSNIGNNYISWYQQFPGTAPK...,AMK70667,AMK70668,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_14_8_3_11,AMK70667_AMK70668,Unique source,Ebola
AMK70669_AMK70670,EVQLLESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLE...,EIVMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKV...,AMK70669,AMK70670,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_7_20_6_3_10,AMK70669_AMK70670,Unique source,Ebola
AMK70671_AMK70672,QVQLVQSGAEVTKPGASVKVSCKASGYTFTTYYMHWVRQAPGQGLE...,DIVMTQSPATLSLSPGERATLSCRASQSVSGYLAWYQQKPGQAPRL...,AMK70671,AMK70672,"immunoglobulin heavy chain V region, partial [...","immunoglobulin heavy chain V region, partial [...",Homo sapiens,"Walker,L.M.",Rapid and large-scale isolation of potent neut...,27-FEB-2016,8_8_17_6_3_11,AMK70671_AMK70672,Unique source,Ebola


In [52]:
plab[plab['targets_mentioned'] == 'Ebola']['reference_title'].value_counts()

Longitudinal Analysis of the Human B Cell Response to Ebola Virus Infection                                                                               883
Rapid and large-scale isolation of potent neutralizing antibodies from a survivor of the 2014 Ebola virus outbreak                                        321
Polyclonal and convergent antibody response to Ebola virus vaccine rVSV-ZEBOV                                                                             155
Monoclonal antibodies and cocktails for treatment of Ebola infections                                                                                      18
The evolution and determinants of neutralization of potent head-binding antibodies against Ebola virus                                                     12
Convergence of a common solution for broad ebolavirus neutralization by glycan cap-directed human antibodies                                               12
Structural Basis for a Convergent Immune Response ag